In [ ]:
# Есть список продаж где то за пол года :

# 1) Код товара
# 2) Сумма скидки - скидка при покупке
# 3) Сумма бонуса - начисленно бонусов при покупке
# 4) Количество - количество едениц проданного товара
# 5) Код торгового автомата - уникальный код торвого автомата / витрины
# 6) Дата и время покупки
# Есть таблица транзакций торговых автоматов.
# Мне нужно построить почасовой прогноз на месяц вперед. Хочу прогнозировать объем продаж по каждому конкретному коду товара.
# Я новичок в анализе данных. Подскажите куда смотреть - буду рад любой помощи.

# p.s. Интересно как вообще такого рода задачи решаются. Это будет простая регрессия по каждому из кодов товара ?

In [ ]:
# Решил отказаться от почасового прогноза и прибегнуть к дневному прогнозу.
# Прогноз продаж продуктов конкретной витрины на N дней

# Шаги:

# Импортирование библиотек
# Загрузить данные

# Обработка данных:
# Создание векторов для ID продукта и витрины.
# Группировка(Суммирование продаж) продаж по уникальному набору витрина-категория-товар на каждый.
# Добавление продаж за последний день на каждый уникальный набор витрина-категория-товар.
# Добавление продаж(средний) за последние 2 наблюдения на каждый уникальный набор витрина-категория-товар.
# Создание словаря витрина-категория-товар-стоимост_товара
# Генерация данных для прогноза
# Преобразование данных в соответствующие тип данных для обучение
# Разделение данных для тренировки и валидации модели
# Внесение данных и обучение
# Прогноз и проверка валидации

In [ ]:
# Импортирование библиотек:

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from xgboost import plot_importance

import time
import sys
import gc
import pickle
import random

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

sys.version_info

In [ ]:
# Загрузка данных

xl = pd.ExcelFile("source.xlsx")
sheets = xl.sheet_names
df = xl.parse(sheets[0])
df

In [ ]:
# Обработка данных

# Для начала надо преобразовать дату в порядковое число, то есть все наблюдения отсортировать по возрастанию даты
# и перевести дни в числовой вектор. А потом нормализовать год и порядковый номер дня.
# Так как есть промежуточные id товаров и магазинов, то надо их тоже преобразовать в порядковый и уникальный номер.

In [ ]:
# Преобразование 'date' в правильный 'datetime' формат
# Извлечение дней, месяцев и годов из 'date'
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Извлечение день недели для каждой даты
df['dayofweek'] = df['date'].dt.dayofweek


# Преобразование первоначальных ID товара, магазина и года(чтобы получить потом порядковый номер дня) в нормализованный вектор [0-max]
shelf_le = preprocessing.LabelEncoder()
shelf_le.fit(df['shelf_id'].unique())
df['shelf_id_encoded'] = shelf_le.transform(df['shelf_id'])

product_le = preprocessing.LabelEncoder()
product_le.fit(df['product_id'].unique())
df['product_id_encoded'] = product_le.transform(df['product_id'])

year_le = preprocessing.LabelEncoder()
year_le.fit(df['year'].unique())
df['year_encoded'] = year_le.transform(df['year'])

# Генерация порядкого номера дня для каждой даты
df['day_block_num'] = df.apply(lambda x: int(x['date'].strftime('%j')), axis=1)
df['day_block_num'] = df['day_block_num'] + 365 * df['year_encoded']

# Генерация порядкого номера месяца для каждой даты
df['month_block_num'] = df['month'] + 12 * df['year_encoded'] - 1

# Генерация нормализованного порядкого номера дня для каждой даты
day_block_num_le = preprocessing.LabelEncoder()
day_block_num_le.fit(df['day_block_num'].unique())
df['day_block_num_encoded'] = day_block_num_le.transform(df['day_block_num'])

# Сортировка данных по нормализованному порядковому номеру дня
df = df.sort_values(by='day_block_num_encoded', ascending=True)

df.head()

In [ ]:
# Группировка

# Группировка продаж по уникальной группе витрина-категория-товар на каждый день.
# Данные были сгенерированы в момент проведение наблюдения, то есть с точностью до секунды.
# Значит, в день могло произойти несколько наблюдений по одному продукту конкретной категории и конкретной витрины.
# Так как нас интересует дневной прогноз, то можно сгруппировать по дням.

# Стоимость товара может изменится за день, поэтому следует выбрать среднее значение за весь день в процессе группировки
# и проинициализировать значения стоимостей товаров.

In [ ]:
# Группировка данных по столбцам 'day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded' суммирую 'count'
grouped_count = df.groupby(['day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded'])['count'].sum()

# Группировка данных по столбцам 'day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded' извлекаю среднее значение 'price'
grouped_price = df.groupby(['day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded'])['price'].mean()

# Преобразование полученных группировок в формат DataFrame
multiindex_count = grouped_count.index.to_frame()
multiindex_price = grouped_price.index.to_frame()
temp_df = pd.DataFrame(multiindex_count[['day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded']].values, columns=['day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded'])
temp_df['count'] = grouped_count.values
temp_df['price'] = grouped_price.values

df = temp_df.copy(True)
df

In [ ]:
# Добавление продаж за последний день на каждую группу витрина-категория-товар

tdf = df.copy(True)
tdf.sort_values(by='day_block_num_encoded', ascending=False)
tdf.reset_index(drop=True, inplace=True)

tdf['lag_count_1'] = [0] * len(tdf)
size = len(tdf.index)
for ind in tdf.index:
    shelf_id = tdf.iloc[ind]['shelf_id_encoded']
    cat_id = tdf.iloc[ind]['cat_id']
    product_id = tdf.iloc[ind]['product_id_encoded']
    day_block_num_encoded = tdf.iloc[ind]['day_block_num_encoded']
    _temp_df = tdf[(tdf['shelf_id_encoded'] == shelf_id) & (tdf['cat_id'] == cat_id) & (tdf['product_id_encoded'] == product_id)
                      & (tdf['day_block_num_encoded'] < day_block_num_encoded)]
    _temp_df.sort_values(by='day_block_num_encoded', ascending=False)

    if len(_temp_df) != 0:
        tdf.at[ind, 'lag_count_1'] = _temp_df.iloc[0]['count']
df = tdf.copy(True)
df

In [ ]:
# Создание словаря витрина-категория-товар-стоимость_товара

    shelves_unique = df['shelf_id_encoded'].unique().tolist()
        shelves_unique.sort()

        shelf_dict = dict()
        for shelf in shelves_unique:
            shelf_df = df[df['shelf_id_encoded'] == shelf]

            categories = shelf_df['cat_id'].unique().tolist()
            cat_product_dict = dict()
            for category in categories:
                product_price = dict()
                products = shelf_df['product_id_encoded'].unique().tolist()
                products.sort()

                product_grouped = shelf_df.groupby(['product_id_encoded'])['price'].max()
                multiindex_price = product_grouped.index.to_frame()
                temp_df = pd.DataFrame(multiindex_price[['product_id_encoded']].values, columns=['product_id_encoded'])
                temp_df['price'] = product_grouped.values
                temp_df['product_id_encoded'] = [int(x) for x in temp_df['product_id_encoded']]
                temp_df.sort_values(by='price', ascending=True)

                for ind in temp_df.index:
                    product_price[temp_df.iloc[ind]['product_id_encoded']] = temp_df.iloc[ind]['price']

                cat_product_dict[category] = product_price
            shelf_dict[shelf] = cat_product_dict

    **Генерация данных для прогноза для каждого уникального набора  витрина-категория-товар-цена
    **

    # столбцы которые будут вносится в модель 
    test_columns = ['day_block_num_encoded', 'dayofweek', 'shelf_id_encoded', 'cat_id', 'product_id_encoded', 'price', 'lag_count_1', 'lag_count_2']
    test_df = pd.DataFrame(columns=test_columns)
    dayofweek = 0

In [ ]:
# Разделение данных для тренировки и валидации модели

# Оставляем последние 30 дней данных для валидации модели, остальные вносятся как тренировочные
# Последние день 'date_block_num_encoded' = 539
X_train = df[df['day_block_num_encoded'] < 509].drop(['count'], axis=1)
Y_train = df[df['day_block_num_encoded'] < 509]['count']

# Последние 30 дней -> 509-539
# Валидационые данные
X_valid = df[(df['day_block_num_encoded'] >= 509) & (df['day_block_num_encoded'] <= 539)].drop(['count'], axis=1)
Y_valid = df[(df['day_block_num_encoded'] >= 509) & (df['day_block_num_encoded'] <= 539)]['count']

# Данные для прогноза.
X_predict = test_df

In [ ]:
# Внесение данных в модель и обучение

 model = XGBRegressor(
        max_depth=8,
        n_estimators=1000,
        min_child_weight=300, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3,    
        seed=42)

    model.fit(
        X_train, 
        Y_train, 
        eval_metric="rmse", 
        eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
        verbose=True, 
        early_stopping_rounds = 50)

**Прогноз и проверка валидации**

# Прогноз для валидации модели
Y_valid_predicted = model.predict(X_valid)

# Прогноз на DAYS_TO_PREDICT дней
Y_predict = model.predict(X_predict)

predicted_N_days_df = X_predict.copy(True)
predicted_N_days_df['predicted'] = pd.Series(Y_predict)
predicted_N_days_df

In [ ]:
# Сравнение прошлых данных с прогнозом на малом объеме данных и влияние факторов